In [1]:
import os
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import matplotlib.pylab as plt

%matplotlib inline

from urllib.request import urlopen
from pathlib import Path
from collections import defaultdict
from random import choice, sample, shuffle

In [2]:
red_lit = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_LIT.txt')
red_apms = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_AP-MS.txt')
red_bin = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_Y2H.txt')

In [3]:
esenciales = open('./Datos/Essential_ORFs_paperHe2.txt')
lista_esenciales = []

for line in esenciales:
    lista_esenciales.append(line.rstrip('\n'))

In [4]:
def nodos_esenciales(red, lista_atr = lista_esenciales, atributo = 'Esencial'):
    
    dict_vacio = dict()
    nx.set_node_attributes(red, dict_vacio.fromkeys(list(red.nodes()),0), atributo )
    
    for nodo in lista_atr:
        try:
            red.nodes[nodo][atributo] = 1
        except KeyError:
            continue
            
    return None

In [5]:
nodos_esenciales(red_lit)
nodos_esenciales(red_bin)
nodos_esenciales(red_apms)

In [6]:
def nodosEsenciales(red, esenciales = lista_esenciales, atributo = 'Esencial'):
    red_aux = red.copy()
    red_aux.remove_nodes_from(list(nodosNoEsenciales(red).nodes()))
    return red_aux.nodes()

In [7]:
def nodosNoEsenciales(red, esenciales = lista_esenciales):
    red_aux = red.copy()
    red_aux.remove_nodes_from(esenciales)
    return red_aux

In [8]:
def sacarEsenciales(red):
    red_aux = red.copy()
    GC = max(nx.connected_component_subgraphs(red_aux), key=len)
    n = GC.number_of_nodes()
    red_aux.remove_nodes_from(list(nodosEsenciales(red)))
    GC = max(nx.connected_component_subgraphs(red_aux), key=len)
    m = GC.number_of_nodes()
    return m/n

In [9]:
sacarEsenciales(red_lit)

0.281121187139324

In [149]:
def sampleNodos(red,dic1,dic2,k,q,dif1=0):
    if dif1 == 0:
        dif1 = dic1[k][1]
    try:
        dif2 = dic1[q][0] - dif1
        sample_nodos = sample(dic2[q],dif1)
        red.remove_nodes_from(sample_nodos)
        if dif2 <= 0:
            return None
        else:
            sampleNodos(red,dic1,dic2,k,q-1,dif1=dif2)
            
    except:
        sampleNodos(red,dic1,dic2,k,q-1,dif1=0)

In [228]:
def sampleNodos2(red, no_ese_grado, lista_grados, indice, tot):
    # grados es una lista que me dice todos los grados que tiene el grafo
    # indice me dice en que grado de la lista estoy parado
    
    # mientras no remueva todos los nodos que me piden
    cont = 0
    acum = 0
    grado = lista_grados[indice]
    # miro los nodos no esenciales con ese grado
    nodos_grado = no_ese_grado[grado]
    while cont < tot:
        if nodos_grado:
            # elijo un nodo no esencial y lo saco de la lista de esos nodos
            shuffle(nodos_grado)
            nodo = nodos_grado.pop()
            # elimino el nodo del grafo
            red.remove_node(nodo)
            cont += 1
            acum += 1
            # para chequear si hizo todos los pasos o requirio pedir nodos
            if cont == tot-1:
                print('Hizo todos los pasos')
            # actualizo la lista de nodos no esenciales
            no_ese_grado[grado] = nodos_grado
        else:
            print(grado)
            if cont == tot:
                
                return red, no_ese_grado, acum
            else:
                try: 
                    no_ese_grado[indice + 1]# and indice < len(lista_grados):
                    # Si no hay mas nodos para sacar con ese grado miro el grado que sigue
                    red, no_ese_grado, acum_aux = sampleNodos2(red, no_ese_grado, lista_grados, indice + 1, tot-cont)
                    cont = tot
                    acum += acum_aux
                    print('Se metio en la recursión para arriba')
                except: 
                    # Si para arriba no hay mas nodos, me muevo para abajo:
                    red, no_ese_grado, acum_aux = sampleNodos2(red, no_ese_grado, lista_grados, indice - 1, tot-cont)
                    cont = tot
                    acum += acum_aux
                    print('Se metio en la recursión para abajo')

    return red, no_ese_grado, acum

In [ ]:
def sampleNodos3(red, no_ese_grado, lista_grados, indice, tot, removidos = 0):
    # grados es una lista que me dice todos los grados que tiene el grafo
    # indice me dice en que grado de la lista estoy parado
    
    # mientras no remueva todos los nodos que me piden
    
    # Cosas que no cambien
    indice donde estoy parado
    cantidad de nodos que tengo que remover
    cantidad de nodos que removi
    
    cont = 0
    n = len(lista_grados)
    grado = lista_grados[indice]
    # miro los nodos no esenciales con ese grado
    nodos_grado = no_ese_grado[grado]
    while cont < tot:
        if nodos_grado:
            # elijo un nodo no esencial y lo saco de la lista de esos nodos
            shuffle(nodos_grado)
            nodo = nodos_grado.pop()
            # elimino el nodo del grafo
            red.remove_node(nodo)
            cont += 1
            removidos += 1
            # para chequear si hizo todos los pasos o requirio pedir nodos
            if cont == tot-1:
                print('Hizo todos los pasos')
            # actualizo la lista de nodos no esenciales
            no_ese_grado[grado] = nodos_grado
        else:
            salir = 0
            print(grado)
            i = indice #(donde estoy parado)
            if i = len(lista_grados)-1:
                salir = 1
            elif salir == 0:
                for ind,gr in zip(range(i+1,len(lista_grados),-1),lista_grados[i+1:]):
                    if no_ese_grado[gr]:
                        red, no_ese_grado, removidos = sampleNodos3(red, no_ese_grado, lista_grados,
                                                                    ind, tot-cont, removidos)
                        salir = 1
                        break
                if salir != 1:
                    
                else:

                    for ind,gr in zip(range(i,0),lista_grados[:i]):
                        if no_ese_grado[gr]:
                            red, no_ese_grado, removidos = sampleNodos3(red, no_ese_grado, lista_grados,
                                                                        ind, tot-cont, removidos)
                            salir = 1
                            break
           
    return red, no_ese_grado, removidos

In [10]:
def randomNoEsenciales(red):
    red_aux = red.copy()
    n = red.number_of_nodes()
    
    degs = sorted(dict(red.degree()).values(), reverse=True)
    nodos = dict(red.nodes(data=True))
    grados = {d: [0,0] for d in degs}
    noEsencXGrado = defaultdict(list)
    for nodo, deg in red.degree():
        grados[deg][0] += nodos[nodo]['Esencial']
        grados[deg][1] += 1-nodos[nodo]['Esencial']
        if (not nodos[nodo]['Esencial']):
            #print(nodos)
            noEsencXGrado[deg].append(nodo)
    grad = sorted(list(grados.keys()))
    
    # almacenamos las cantidades de nodos en el dataframe para que sea mas facil manipular
    df_nodos = pd.DataFrame(grados).T
    df_nodos.columns = ['esenciales', 'no esenciales']
    df_nodos['diferencia'] = df_nodos['no esenciales'] - df_nodos['esenciales']   
    
    #list(df[df.esenciales>=1].index)
    #print(grad)
    #print(noEsencXGrado)
    
    diferencias = df_nodos.diferencia
    
    print(df_nodos)
    prohibidos = []
    for grado, ind in zip(grad, range(len(grad))):
        # Para ese grado, saco tantos nodos no esenciales como nodos esenciales hay con ese grado
        # Cantidad de nodos a remover para un grado fijo
        tot_es = df_nodos.loc[grado].esenciales
        
        print("Total a sacar {}".format(tot_es))
        print("Grado: {}, Indice: {}".format(grad[ind],ind))
        red_aux, noEsencXGrado, prohibidos = remover_nodos_no_esenciales2(red_aux, ind, grad, tot_es,
                                                                          noEsencXGrado, prohibidos)
        #print("removidos {}".format(removidos))
    return red_aux.number_of_nodes()/n, grados, grad, df_nodos

In [11]:
from numpy.random import choice

In [38]:
def remover_nodos_no_esenciales2(red, indice, lista_grados, tot_remover, lista_no_esenciales_grado,
                                prohibidos, removidos = 0):
    
    #prohibidos es una lista vacia en el primer caso
    
    grado = lista_grados[indice]
    # miro los nodos no esenciales con ese grado
    #nodos_grado = lista_no_esenciales_grado[grado]
    n = len(lista_grados)
    i = 0
    while removidos < tot_remover:
        m = len(lista_no_esenciales_grado[grado])
        if m > 0:
            
            nodos_a_remover = np.random.choice(lista_no_esenciales_grado[grado], min(m, tot_remover - removidos),
                                               replace = False)
            # Filtro por las dudas
            nodos = [nodo for nodo in nodos_a_remover if nodo not in prohibidos]
            red.remove_nodes_from(nodos)
            for nodo in nodos:
                lista_no_esenciales_grado[grado].remove(nodo)
            if len(nodos) == 0:
                i += 1
                lista_no_esenciales_grado[grado] = lista_no_esenciales_grado[lista_grados[min(indice+i,
                                                                                          n-1)]] + lista_no_esenciales_grado[lista_grados[max(indice-i, 0)]]
            removidos += len(nodos)
            # Agrego los prohibidos a la lista de nodos
            prohibidos = prohibidos + nodos
            
            print('saco {}'.format(min(m, tot_remover, len(nodos))))
            #print(lista_no_esenciales_grado[grado])
            
        else:
            i += 1
            lista_no_esenciales_grado[grado] = lista_no_esenciales_grado[lista_grados[min(indice+i, n-1)]] + lista_no_esenciales_grado[lista_grados[max(indice-i, 0)]]
            #nodos = np.choice(lista_no_esenciales_grado[grado], min(m, tot_remover), replace = False)
            #red.remove_node(nodo)
            #lista_no_esenciales_grado[grado].remove(nodos)

    # para un chequeo

    print('prohibi: {}'.format(len(prohibidos)))
        
    return red, lista_no_esenciales_grado, prohibidos

In [54]:
df.esenciales.values.sum()

625

In [53]:
k, gr, acum, df = randomNoEsenciales(red_lit)

    esenciales  no esenciales  diferencia
1          139            391         252
2           97            174          77
3           91            107          16
4           62             58          -4
5           45             54           9
6           38             30          -8
7           37             25         -12
8           20             14          -6
9           25             12         -13
10          11             10          -1
11          14              4         -10
12           8              5          -3
13           9              7          -2
14           5              5           0
15           2              1          -1
16           3              7           4
17           1              1           0
18           1              0          -1
19           2              2           0
21           1              1           0
22           1              0          -1
23           2              0          -2
24           2              3     

In [233]:
def get_next_index(lista, indice):
    
    i = len(lista)
    for elem, ind in zip(lista[indice:], range(indice,len(lista))):
        if elem > 0: 
            i = ind
            break
    j = -1
    for elem, ind in zip(lista[:indice], range(0,indice)):
        if elem > 0: 
            j = ind
          
    return j, i

In [245]:
a = [-1,-20,0,-1,-1,-2,-3,4,-4,-15,-1]
ind = 5
get_next_index(a,ind)

(7, -1)

In [ ]:
def auxiliar(total, donde=indice, i_prev):
    
    # dif =[0,-1,3,4,6,-3] me dice cuantos puedo sacar basicamente de cada grado (no esen - esenciales)
    # donde = el grado / indice de la lista
    removi = 0
    while removi < total:
        
        if dif[indice] >= total:
            remover(total o lo que pueda)
            actualizo cantidad removidos
            actualizo diferencias
        else:
            
            i_prev, i_sig = indices de donde puedo sacar
            if i_sig no es el ultimo and indicador == 0:
                
                auxiliar(total - removidos, i_sig)
                #sino ya agote los siguientes y vuelvo al original
                
            else:
                auxiliar(total-removidos, indice,)
    
    
    return(donde, cuantos)

In [62]:
df = pd.DataFrame(gr).T
df.sort_index(axis=0, ascending=False, inplace=True)
df.columns=['esenciales', 'no esenciales']
list(df[df.esenciales>=1].index)

In [65]:
df

,esenciales,no esenciales
91,1,0
82,0,1
81,1,0
52,0,1
46,1,0
42,0,1
37,0,1
32,0,1
30,0,1
29,1,3


In [67]:
list(df[df.esenciales>=1].index)

[91, 81, 46, 29, 21, 18, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [191]:
lista = [1,2,3,4]
a = lista[-2:].reverse()

In [226]:
print(list(range(4,0,-1)))

[4, 3, 2, 1]
